In [1]:
# ----------------------------------------------------------------------------
# Project     : mrs - multi-turn response selection
# Created By  : Eungis
# Team        : AI Engineering
# Created Date: 2023-11-30
# Updated Date: 2023-12-07
# Purpose     : Make data_loader for loading data
# version     : 0.0.1
# ---------------------------------------------------------------------------

In [12]:
import logging
import pandas as pd
from importlib import reload  # Not needed in Python 2

reload(logging)
logging.basicConfig(
    format="%(message)s",
    level=logging.DEBUG,
)

DATA_ROOT = "../data/"
logger = logging.getLogger()
data = pd.read_csv(DATA_ROOT + "smilestyle_dataset.tsv", sep="\t")

In [31]:
cols = data.columns.tolist()
logger.debug(f"Columns: {cols}")
# use formal conversational data
data = data[["formal"]]

data["group"] = data["formal"].isnull().cumsum()
n_sessions = data["group"].iat[-1] + 1
logger.debug(f"Number of groups: {n_sessions}")

sessions = []
groups = data.groupby("group", as_index=False, group_keys=False)

for i, group in groups:
    session = group.dropna()["formal"].tolist()
    sessions += [session]

assert n_sessions == len(sessions)

Columns: ['formal', 'group']
Number of groups: 236


In [38]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
logger.debug(f"Special tokens: {tokenizer.special_tokens_map}")
logger.debug(f"EOS token & SEP token: {tokenizer.eos_token} / {tokenizer.sep_token}")
special_tokens = {"sep_token": "<SEP>"}
tokenizer.add_special_tokens(special_tokens)

https://huggingface.co:443 "HEAD /klue/roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
Special tokens: {'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
EOS token & SEP token: [SEP] / [SEP]


1